In [13]:
!pip install --upgrade pip
!pip install transformers
!pip install torch
!pip install sentencepiece

     |████████████████████████████████| 1.1 MB 1.6 MB/s eta 0:00:01


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json

In [2]:
data_dev_file = 'data/MCL-WiC/dev/multilingual/dev.ru-ru.data'

In [3]:
with open(data_dev_file) as f_data:
    data_samples = json.load(f_data)
    
data_samples[2]

{'id': 'dev.ru-ru.2',
 'lemma': 'требовать',
 'pos': 'VERB',
 'sentence1': 'В каждой конкретной стране эта проблема имеет свои особенности и поэтому требует особого подхода.',
 'sentence2': 'Все это порождает ситуацию, которая требует, чтобы международное сообщество сотрудничало и стало взаимозависимым в своих усилиях по обеспечению безопасности и прогресса.',
 'start1': '73',
 'end1': '80',
 'start2': '36',
 'end2': '43'}

In [30]:
from transformers import MarianMTModel, MarianTokenizer
import torch

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
src_text = [
    'Регистрация в суде представляет собой лишь уведомление, а не получение разрешения.',
    'Касаясь апелляции, авторы заявляют, что адвокат, представлявший их в апелляционном суде, не проводил с ними никаких консультаций до слушания апелляции, на котором авторы не присутствовали.',
    'В каждой конкретной стране эта проблема имеет свои особенности и поэтому требует особого подхода.',
    'Все это порождает ситуацию, которая требует, чтобы международное сообщество сотрудничало и стало взаимозависимым в своих усилиях по обеспечению безопасности и прогресса.'
]

model_name = 'Helsinki-NLP/opus-mt-ru-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)

text_batch = tokenizer.prepare_seq2seq_batch(src_text, return_tensors="pt").to(device)
translated = model.generate(**text_batch)
tgt_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

In [6]:
tgt_text

['Registration in court is only a notice, not a permit.',
 'With regard to the appeal, the authors submit that the lawyer who represented them in the Court of Appeal did not consult them at all until the appeal hearing, at which the authors were not present.',
 'In each particular country, the problem had its own characteristics and therefore required a special approach.',
 'All of this creates a situation that requires the international community to cooperate and become interdependent in its efforts to ensure security and progress.']

In [7]:
data_samples = data_samples[:10] # TODO - delete

In [8]:
from tqdm.auto import tqdm

In [9]:
batch_size = 16

for start_ind in tqdm(range(0, len(data_samples), batch_size)):
    batch_sents = []
    
    for i in range(batch_size):
        if start_ind + i >= len(data_samples):
            break
        current_sample = data_samples[start_ind+i]
        batch_sents.append(current_sample['sentence1'])
        batch_sents.append(current_sample['sentence2'])
        
    translated = model.generate(**tokenizer.prepare_seq2seq_batch(batch_sents, return_tensors="pt"))
    trans_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    
    for i in range(batch_size):
        if start_ind + i >= len(data_samples):
            break
        current_sample = data_samples[start_ind+i]
        current_sample['trans_sentence1'] = trans_texts[2 * i]
        current_sample['trans_sentence2'] = trans_texts[2 * i + 1]

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
data_samples[:5]

[{'id': 'dev.ru-ru.0',
  'lemma': 'представлять',
  'pos': 'VERB',
  'sentence1': 'Регистрация в суде представляет собой лишь уведомление, а не получение разрешения.',
  'sentence2': 'Касаясь апелляции, авторы заявляют, что адвокат, представлявший их в апелляционном суде, не проводил с ними никаких консультаций до слушания апелляции, на котором авторы не присутствовали.',
  'start1': '19',
  'end1': '31',
  'start2': '49',
  'end2': '63',
  'trans_sentence1': 'Registration in court is only a notice, not a permit.',
  'trans_sentence2': 'With regard to the appeal, the authors submit that the lawyer who represented them in the Court of Appeal did not consult them at all until the appeal hearing, at which the authors were not present.'},
 {'id': 'dev.ru-ru.1',
  'lemma': 'представлять',
  'pos': 'VERB',
  'sentence1': 'Регистрация в суде представляет собой лишь уведомление, а не получение разрешения.',
  'sentence2': 'В таких случаях размер пособия по безработице представляет собой разниц

In [14]:
data_samples[-5:]

[{'id': 'dev.ru-ru.138',
  'lemma': 'убийство',
  'pos': 'NOUN',
  'sentence1': 'Европейский союз напоминает об общей позиции его Совета по Афганистану и об осуждении Европейским союзом убийства иранских дипломатов во время недавних боев в Мазари-Шарифе.',
  'sentence2': 'Ему инкриминируется участие в массовых убийствах, имевших место в Багогве в 1992 году.',
  'start1': '105',
  'end1': '113',
  'start2': '39',
  'end2': '48',
  'trans_sentence1': 'The European Union recalls the common position of its Council on Afghanistan and the European Union &apos; s condemnation of the murder of Iranian diplomats during the recent fighting in Mazar-i-Sharif.',
  'trans_sentence2': 'He is accused of involvement in the massacres that took place in Bagogwe in 1992.'},
 {'id': 'dev.ru-ru.139',
  'lemma': 'убийство',
  'pos': 'NOUN',
  'sentence1': 'Европейский союз напоминает об общей позиции его Совета по Афганистану и об осуждении Европейским союзом убийства иранских дипломатов во время недавних б